In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Datos2")

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree  import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import RandomOverSampler

In [8]:
data=pd.read_csv('diabetes_binary_health_indicators_BRFSS2015.csv')
data = data_clean(data)

Limpieza de datos que ayudaran al momento de preprosesar los datos

In [7]:
def data_clean(dataframe):

    bins = [0, 18.5, 24.9, 29.9, float('inf')]
    labels = [1,2,3,4]

    df = (dataframe
         .drop_duplicates()
         .assign(BMI_bins=dataframe.BMI.pipe(pd.cut, bins=bins, labels=labels))
         .reset_index(drop=True)
         .astype({'Diabetes_binary':'uint8',
     'HighBP':'uint8',
     'HighChol':'uint8',
     'CholCheck':'uint8',
     'BMI':'uint8',
     'Smoker':'uint8',
     'Stroke':'uint8',
     'HeartDiseaseorAttack':'uint8',
     'PhysActivity':'uint8',
     'Fruits':'uint8',
     'Veggies':'uint8',
     'HvyAlcoholConsump':'uint8',
     'AnyHealthcare':'uint8',
     'NoDocbcCost':'uint8',
     'GenHlth':'uint8',
     'MentHlth':'uint8',
     'PhysHlth':'uint8',
     'DiffWalk':'uint8',
     'Sex':'uint8',
     'Age':'uint8',
     'Education':'uint8',
     'Income':'uint8',
     'BMI_bins':'uint8'})


         )
    return df

# Preprosesamiento

In [9]:
# df.shape
print(f"Num rows: {len(data)}")
print(f"Num columns: {len(data.columns)}")

Num rows: 229474
Num columns: 23


imprimimos los valore unicos que tenemos dentro de cada columna.

In [10]:
for column in data.columns:
    unique_values = data[column].unique()
    print("Unique values for column", column, ":")
    print(unique_values)
    print()


Unique values for column Diabetes_binary :
[0 1]

Unique values for column HighBP :
[1 0]

Unique values for column HighChol :
[1 0]

Unique values for column CholCheck :
[1 0]

Unique values for column BMI :
[40 25 28 27 24 30 34 26 33 21 23 22 38 32 37 31 29 20 35 45 39 19 47 18
 36 43 55 49 42 17 16 41 44 50 59 48 52 46 54 57 53 14 15 51 58 63 61 56
 74 62 64 66 73 85 60 67 65 70 82 79 92 68 72 88 96 13 81 71 75 12 77 69
 76 87 89 84 95 98 91 86 83 80 90 78]

Unique values for column Smoker :
[1 0]

Unique values for column Stroke :
[0 1]

Unique values for column HeartDiseaseorAttack :
[0 1]

Unique values for column PhysActivity :
[0 1]

Unique values for column Fruits :
[0 1]

Unique values for column Veggies :
[1 0]

Unique values for column HvyAlcoholConsump :
[0 1]

Unique values for column AnyHealthcare :
[1 0]

Unique values for column NoDocbcCost :
[0 1]

Unique values for column GenHlth :
[5 3 2 4 1]

Unique values for column MentHlth :
[18  0 30  3  5 15 10  6 20  2 25  1

In [11]:
#Function to view the value counts for the whole data
def v_counts(dataframe):
    for i in dataframe :
        print(dataframe[i].value_counts())
        print("_____________________________________________________________________________")
v_counts(data)

0    194377
1     35097
Name: Diabetes_binary, dtype: int64
_____________________________________________________________________________
0    125214
1    104260
Name: HighBP, dtype: int64
_____________________________________________________________________________
0    128129
1    101345
Name: HighChol, dtype: int64
_____________________________________________________________________________
1    220176
0      9298
Name: CholCheck, dtype: int64
_____________________________________________________________________________
27    21514
26    17775
24    16497
28    14914
25    14793
      ...  
85        1
91        1
86        1
90        1
78        1
Name: BMI, Length: 84, dtype: int64
_____________________________________________________________________________
0    122585
1    106889
Name: Smoker, dtype: int64
_____________________________________________________________________________
0    219190
1     10284
Name: Stroke, dtype: int64
____________________________________________

### Verificacion de datos nulos

In [12]:
data.isnull().sum()

Diabetes_binary         0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
BMI_bins                0
dtype: int64

Como podemos ver no existen valores nulos dentro de los campos


# Verificacion de datos Duplicados

In [13]:
data.duplicated().sum()

0

In [14]:
data.drop_duplicates(inplace=True)

In [15]:
data.duplicated().sum()

0

# Clasificador
Para este seccion hemos seleccionaod el calsificador de arbol de desicion

Para este proceso lo que primero vamos a hacer es Scalar las caracteristicas(X) y separar el objetivo (Y)

In [18]:
X = data.drop(['Diabetes_binary'],axis=1)
y = data['Diabetes_binary']


oversampler = RandomOverSampler()
X_resampled, y_resampled = oversampler.fit_resample(X, y)



In [22]:
scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

Verificamos

In [21]:
scaled_features = scaler.transform(X)
X = pd.DataFrame(scaled_features,columns=data.columns[1:])
X.head(10)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,BMI_bins
0,1.095892,1.124404,0.205499,1.666251,1.070908,-0.216606,-0.339478,-1.657077,-1.257701,0.508445,...,-0.320063,2.251897,1.877538,1.140089,2.093694,-0.884764,0.295669,-0.986660,-1.380208,1.114127
1,-0.912499,-0.889360,-4.866203,-0.543144,1.070908,-0.216606,-0.339478,0.603472,-1.257701,-1.966783,...,3.124389,0.373892,-0.454786,-0.517213,-0.477625,-0.884764,-0.350650,1.027465,-2.335828,-0.105855
2,1.095892,1.124404,0.205499,-0.101265,-0.933787,-0.216606,-0.339478,-1.657077,0.795102,-1.966783,...,3.124389,2.251897,3.432420,2.797390,2.093694,-0.884764,0.295669,-0.986660,1.008840,-0.105855
3,1.095892,-0.889360,0.205499,-0.248558,-0.933787,-0.216606,-0.339478,0.603472,0.795102,0.508445,...,-0.320063,-0.565110,-0.454786,-0.517213,-0.477625,-0.884764,0.941989,-1.993722,0.053221,-0.105855
4,1.095892,1.124404,0.205499,-0.690437,-0.933787,-0.216606,-0.339478,0.603472,0.795102,0.508445,...,-0.320063,-0.565110,-0.066065,-0.517213,-0.477625,-0.884764,0.941989,0.020402,-0.902399,-1.325838
5,1.095892,1.124404,0.205499,-0.543144,1.070908,-0.216606,-0.339478,0.603472,0.795102,0.508445,...,-0.320063,-0.565110,-0.454786,-0.296239,-0.477625,1.130245,0.618829,1.027465,1.008840,-0.105855
6,1.095892,-0.889360,0.205499,0.193321,1.070908,-0.216606,-0.339478,-1.657077,-1.257701,-1.966783,...,-0.320063,0.373892,-0.454786,1.029602,-0.477625,-0.884764,0.295669,1.027465,0.531031,1.114127
7,1.095892,1.124404,0.205499,-0.543144,1.070908,-0.216606,-0.339478,0.603472,-1.257701,0.508445,...,-0.320063,0.373892,-0.454786,-0.517213,2.093694,-0.884764,0.941989,-0.986660,-0.902399,-0.105855
8,1.095892,1.124404,0.205499,0.193321,1.070908,-0.216606,2.945698,-1.657077,0.795102,0.508445,...,-0.320063,2.251897,3.432420,2.797390,2.093694,-0.884764,0.295669,0.020402,-2.335828,1.114127
9,-0.912499,-0.889360,0.205499,-0.690437,-0.933787,-0.216606,-0.339478,-1.657077,-1.257701,0.508445,...,-0.320063,-0.565110,-0.454786,-0.517213,-0.477625,1.130245,-0.027491,-0.986660,-1.380208,-1.325838


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.20, random_state=0)

In [24]:
model= DecisionTreeClassifier()
model.fit(X_train , y_train)

DecisionTreeClassifier()

In [25]:
print(model.score(X_train , y_train))
print(model.score(X_test, y_test))

0.995864991656029
0.9153708633972554


In [26]:
# Calcular la matriz de confusión

y_pred = model.predict(X_test)

confusion = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(confusion)


# Calcular la precisión del clasificador
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del Clasificador:", accuracy)

Matriz de Confusión:
[[32744  6243]
 [  337 38427]]
Precisión del Clasificador: 0.9153708633972554


* En la clase 0, hay 36,922 instancias que fueron clasificadas correctamente como clase 0, 155 instancias que fueron clasificadas incorrectamente como clase 1 y 1,039 instancias que fueron clasificadas incorrectamente como clase 2.
* En la clase 1, hay 176 instancias que fueron clasificadas incorrectamente como clase 0, 694 instancias que fueron clasificadas correctamente como clase 1 y 36 instancias que fueron clasificadas incorrectamente como clase 2.
* En la clase 2, hay 1,075 instancias que fueron clasificadas incorrectamente como clase 0, 61 instancias que fueron clasificadas incorrectamente como clase 1 y 5,799 instancias que fueron clasificadas correctamente como clase 2.

# Primera ejecucion
Splits: al menos 100 asignaciones, la mediana de la confiabilidad
Académico (primera ejecucion) 80(train)/20(test) – Investigación 50/50 (segunda
ejecución)

In [27]:
accuracy_list=[]
for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.20, random_state=0)
  model= DecisionTreeClassifier()
  model.fit(X_train , y_train)
  y_pred = model.predict(X_test)
  confusion = confusion_matrix(y_test, y_pred)
  print("Matriz de Confusión:")
  print(confusion)
  accuracy = accuracy_score(y_test, y_pred)
  print("Precisión del Clasificador:", accuracy)
  accuracy_list.append(accuracy)
  print("__________________________________________________")
print(accuracy_list)
print(np.mean(accuracy_list))

Matriz de Confusión:
[[32786  6201]
 [  336 38428]]
Precisión del Clasificador: 0.915923910946483
__________________________________________________
Matriz de Confusión:
[[32740  6247]
 [  339 38425]]
Precisión del Clasificador: 0.9152936939717817
__________________________________________________
Matriz de Confusión:
[[32753  6234]
 [  330 38434]]
Precisión del Clasificador: 0.9155766485318517
__________________________________________________
Matriz de Confusión:
[[32733  6254]
 [  334 38430]]
Precisión del Clasificador: 0.9152679708299571
__________________________________________________
Matriz de Confusión:
[[32736  6251]
 [  337 38427]]
Precisión del Clasificador: 0.9152679708299571
__________________________________________________
Matriz de Confusión:
[[32745  6242]
 [  330 38434]]
Precisión del Clasificador: 0.9154737559645535
__________________________________________________
Matriz de Confusión:
[[32719  6268]
 [  330 38434]]
Precisión del Clasificador: 0.9151393551208344
__

# Segunda ejecucion

In [28]:
accuracy_list=[]
for i in range(100):
  X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.50, random_state=0)
  model= DecisionTreeClassifier()
  model.fit(X_train , y_train)
  y_pred = model.predict(X_test)
  confusion = confusion_matrix(y_test, y_pred)
  print("Matriz de Confusión:")
  print(confusion)
  accuracy = accuracy_score(y_test, y_pred)
  print("Precisión del Clasificador:", accuracy)
  accuracy_list.append(accuracy)-
  print("__________________________________________________")
print(accuracy_list)
print(np.mean(accuracy_list))


Matriz de Confusión:
[[77245 20235]
 [ 3825 93072]]
Precisión del Clasificador: 0.8762199231390545
__________________________________________________
Matriz de Confusión:
[[77128 20352]
 [ 3838 93059]]
Precisión del Clasificador: 0.8755511197312439
__________________________________________________
Matriz de Confusión:
[[77125 20355]
 [ 3867 93030]]
Precisión del Clasificador: 0.8753864912000906
__________________________________________________
Matriz de Confusión:
[[77147 20333]
 [ 3823 93074]]
Precisión del Clasificador: 0.8757260375455944
__________________________________________________
Matriz de Confusión:
[[77149 20331]
 [ 3846 93051]]
Precisión del Clasificador: 0.875618000072025
__________________________________________________
Matriz de Confusión:
[[77165 20315]
 [ 3842 93055]]
Precisión del Clasificador: 0.8757208929039958
__________________________________________________
Matriz de Confusión:
[[77195 20285]
 [ 3864 93033]]
Precisión del Clasificador: 0.8757620500367842
__